# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [43]:
# your code here
import pandas as pd

tic_tac_toe = pd.read_csv('tic-tac-toe.csv')
tic_tac_toe

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [44]:
for i in tic_tac_toe.columns:
    tic_tac_toe[i] = tic_tac_toe[i].map({'x':1,'o':0,'b':2, True:0, False:1})

tic_tac_toe

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,0,0,1,0,0,0
1,1,1,1,1,0,0,0,1,0,0
2,1,1,1,1,0,0,0,0,1,0
3,1,1,1,1,0,0,0,2,2,0
4,1,1,1,1,0,0,2,0,2,0
...,...,...,...,...,...,...,...,...,...,...
953,0,1,1,1,0,0,0,1,1,1
954,0,1,0,1,1,0,1,0,1,1
955,0,1,0,1,0,1,1,0,1,1
956,0,1,0,0,1,1,1,0,1,1


In [45]:
input_ttt = tic_tac_toe[['TL', 'TM', 'TR', 'ML', 'MM', 'MR', 'BL', 'BM', 'BR']]
output_ttt = tic_tac_toe['class']

output_ttt

0      0
1      0
2      0
3      0
4      0
      ..
953    1
954    1
955    1
956    1
957    1
Name: class, Length: 958, dtype: int64

In [46]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
input_ttt = scaler.fit_transform(input_ttt)
input_ttt

array([[ 0.1837706 ,  0.10794659,  0.1837706 , ...,  0.1837706 ,
        -1.18471377, -1.17047733],
       [ 0.1837706 ,  0.10794659,  0.1837706 , ..., -1.17047733,
         0.10794659, -1.17047733],
       [ 0.1837706 ,  0.10794659,  0.1837706 , ..., -1.17047733,
        -1.18471377,  0.1837706 ],
       ...,
       [-1.17047733,  0.10794659, -1.17047733, ...,  0.1837706 ,
        -1.18471377,  0.1837706 ],
       [-1.17047733,  0.10794659, -1.17047733, ...,  0.1837706 ,
        -1.18471377,  0.1837706 ],
       [-1.17047733, -1.18471377,  0.1837706 , ..., -1.17047733,
         0.10794659,  0.1837706 ]])

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [47]:
# your code here
from sklearn.model_selection import train_test_split
input_ttt_train, input_ttt_test, output_ttt_train, output_ttt_test = train_test_split(input_ttt, output_ttt)
display(input_ttt_train.shape)
display(output_ttt_train.shape)

(718, 9)

(718,)

In [60]:
import tensorflow as tf


model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(128, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(64, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
model.add(tf.keras.layers.Dense(2, activation=tf.nn.softmax))
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(input_ttt_train, output_ttt_train, epochs=8)

Epoch 1/8
23/23 [==============================] - 0s 12ms/step - loss: 0.6258 - accuracy: 0.6504
Epoch 2/8
23/23 [==============================] - 0s 10ms/step - loss: 0.5350 - accuracy: 0.7382
Epoch 3/8
23/23 [==============================] - 0s 10ms/step - loss: 0.4368 - accuracy: 0.7953
Epoch 4/8
23/23 [==============================] - 0s 9ms/step - loss: 0.3292 - accuracy: 0.8677
Epoch 5/8
23/23 [==============================] - 0s 9ms/step - loss: 0.2493 - accuracy: 0.9095
Epoch 6/8
23/23 [==============================] - 0s 13ms/step - loss: 0.1717 - accuracy: 0.9318
Epoch 7/8
23/23 [==============================] - 0s 10ms/step - loss: 0.1320 - accuracy: 0.9596
Epoch 8/8
23/23 [==============================] - 0s 10ms/step - loss: 0.0752 - accuracy: 0.9777


In [61]:
val_loss, val_acc = model.evaluate(input_ttt_test, output_ttt_test)
print(val_loss)
print(val_acc)

8/8 [==============================] - 0s 8ms/step - loss: 0.1711 - accuracy: 0.9208
0.17114534974098206
0.9208333492279053


In [62]:
model.save('tic-tac-toe.model')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: tic-tac-toe.model/assets


## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [73]:
# your code here
new_model = tf.keras.models.load_model('tic-tac-toe.model')

predictions = new_model.predict(input_ttt_test)


In [95]:
import numpy as np

output_ttt_test.reset_index(inplace=True, drop=True)

display(np.argmax(predictions[0]))
display(output_ttt_test[0])

1

0

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [168]:
# your code here
new_model = tf.keras.models.load_model('tic-tac-toe.model')
opt = tf.keras.optimizers.Adam(learning_rate=0.0013)
new_model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

new_model.fit(input_ttt_train, output_ttt_train, epochs=5)

Epoch 1/5
23/23 [==============================] - 0s 10ms/step - loss: 0.0780 - accuracy: 0.9735
Epoch 2/5
23/23 [==============================] - 0s 9ms/step - loss: 0.0395 - accuracy: 0.9930
Epoch 3/5
23/23 [==============================] - 0s 8ms/step - loss: 0.0239 - accuracy: 0.9986
Epoch 4/5
23/23 [==============================] - 0s 11ms/step - loss: 0.0180 - accuracy: 0.9944
Epoch 5/5
23/23 [==============================] - 0s 10ms/step - loss: 0.0150 - accuracy: 0.9986


In [169]:
val_loss, val_acc = new_model.evaluate(input_ttt_test, output_ttt_test)
print(val_loss)
print(val_acc)

8/8 [==============================] - 0s 7ms/step - loss: 0.0962 - accuracy: 0.9583
0.09617592394351959
0.9583333134651184


In [170]:
new_model.save('tic-tac-toe_perfect.model')

INFO:tensorflow:Assets written to: tic-tac-toe_perfect.model/assets


**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
"""
Adding new layers did not help me to improve the model, 
so I decided not to add more layers and the parameters changed.
For me, put a learning_rate low (0.0013) and a number of epochs = 5
"""